# Extraction de Keywords

## Imports

In [ ]:
import os
import requests
import yake
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Extraire les mots clés d'un document avec Yake

https://github.com/LIAAD/yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:

data_path = "../data/txt/"

files = [f for f in sorted(os.listdir(data_path)) if f.endswith('.txt')]
len(files)




In [ ]:
# Les fichiers
files

In [ ]:
# Changer le nom de la variable 
this_file = files
this_file

In [ ]:
# Initialisation de liste pour stocker les textes
texts = []

# Parcourez la liste des noms de fichiers
for this_file in this_file:
    file_path = os.path.join(data_path, this_file)
    
    # Vérifiez si le fichier existe
    if os.path.exists(file_path):
        # Lisez le contenu du fichier et stockez-le dans la liste
        with open(file_path, 'r', encoding='utf-8') as file:
            file_text = file.read()
            texts.append(file_text)

In [ ]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(file_text)
keywords

In [ ]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

## Faire la même opération sur tous les documents

In [ ]:
for f in sorted(files):
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

## NUAGES DE MOTS

In [ ]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "la", "jusqu'à", "trois", 
       "voilà","le", "du", "afin," "après", "aucun", "aujourd", "assez", "allons", "abord"
       "tellment", "telle", "tienne", "vif", "va", "via", "vers", "vives", "vous", "encore", "tic"
       "tes", "te", "votre", "surtout", "tac", "sur", "vous-memes", "treize","sur", "un", "notre",
       "vais", "trop", "sujet","toc","vont","toi-meme", "tien", "suivant", "mienne", "tels"
       "devant", "téléphone", "devant", "acheter","vol", "achat", "libre", "offre", "bon"
       "pr", "ecr","une", "encore", "suite", "ceux", "celle", "celui", "celui-ci", "mais"
       "ser", "mai", "display", "ou", "dès", "ref", "quelques", "très", "vue", "voir"
       "une", "lot", "tél", "bon", "mme", "après", "fin","notamment", "aussi", "les", "toutes",
       "elles", "ils", "vos", "elle", "celle-la", "tellement", "supérieur", "grace", "pas", "quand",
       "vont", "force", "forte", "egal", "plus", "celle", "deux", "achat", "corpus", "nom", "lorsque",
       "pendant que", "alors que", "eut", "eurent", "chemin", "ensemble", "est", "était", "étaient",
       "vivre", "vit", "allons", "sur", "avant", "mal", "a", "avait", "à", "ici", "parce", "parce que", 
       "hier", "pris", "déjà", "dix", "point", "nouveau", "rue", "reçu", "toute", "tout", "ans",
       "raison", "mis", "art", "huit", "cas", "tant", "mis", "pris", "fort", "fit", "dos", "appel", 
       "ensuite", "voie", "samedi", "tete", "vie", "ira", "peu", "heure", "fils", "rue", "midi", 
       "jamais", "place", "cinq", "raison", "dan", "dans", "car", "ensuite", "oui", "on", "ont", "effet",
       "soir", "dos", "hui", "midi", "dos", "en", "samedi", "peine", "alors", "oui", "fer", "enfin", 
       "titre", "voir", "tant", "tant que", "oui", "jeudi", "ici", "chez", "mois", "cas", "temps", "près",
       "matin", "aucune", "haut", "dernier", "tour", "qne", "matin", "uue", "pourra", "haut", "prix", 
       "donner", "jour", "nuit", "six", "faite", "reste", "main", "puis", "choses", "veut", "seul", 
       "moyen", "prit", "mettre", "dernier", "aussitot", "mieux", "cause", "chaque", "prix", "leurs", 
       "fois", "mit", "seul", "matin", "bonne", "chose", "part", "lieu", "petit", "pouvoir", 
       "veut", "dooner", "rendu", "eau", "nuit", "hotel", "jours", "septembre", "heures", "leurs",
       "fois", "charge", "ordre", "lequel", "presque", "charge", "somme", " faits", "lorsqu", "choses",
       "dimanche", "dessus", "travaux", "yeux", "vient", "heures", "sou", "coup", "etc"]
sw = set(sw)


In [ ]:
print(f"{len(sw)} stopwords:\n {sorted(sw)}")

In [ ]:
# Compter le nombre d'éléments donc fichiers) dans la liste
len(texts)

In [ ]:
# Imprimer tous les caractères du contenu dans les fichiers
#for text in texts:
   # for char in text:
        #print(char, end='')

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'all_2.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(texts))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'all_2.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:1000]

In [ ]:
def clean_text(folder=None):
    if folder is None:
        input_path = f"all_2.txt"
        output_path = f"all_2_clean.txt"
    else:
        input_path = f"{folder}/all_2.txt"
        output_path = f"{folder}/all_2_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'all_2_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:1000]

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(10))

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"all_2_clean.png"))
Image(filename=os.path.join(temp_path, f"all_2_clean.png"))

## RECONNAISSANCE D'ENTITE NOMMEES AVEC SPACY

In [ ]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_md')

In [ ]:
n=100000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 50

In [ ]:
sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:30]:
    print(f"{person} apparait {freq} fois dans le corpus")

Afficher les Lieux et Organisations les plus mentionnées dans le Corpus

In [ ]:
# Analyse du texte  avec SpaCy
doc = nlp(text)

# Extraction et count des lieux (LOC) et les organisations (ORG)
locations = {}
organizations = {}

for ent in doc.ents:
    if ent.label_ == "LOC":
        locations[ent.text] = locations.get(ent.text, 0) + 1
    elif ent.label_ == "ORG":
        organizations[ent.text] = organizations.get(ent.text, 0) + 1

# Trier et afficher les lieux les plus mentionnés
sorted_locations = sorted(locations.items(), key=lambda kv: kv[1], reverse=True)
print("\nLieux les plus mentionnés :")
for location, freq in sorted_locations[:50]:
    print(f"{location} apparait {freq} fois dans le corpus")

# Trier et afficher les organisations les plus mentionnées
sorted_organizations = sorted(organizations.items(), key=lambda kv: kv[1], reverse=True)
print("\nOrganisations les plus mentionnées :")
for organization, freq in sorted_organizations[:50]:
    print(f"{organization} apparait {freq} fois dans le corpus")

## Sentiment analysis avec Textblob-FR

In [ ]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

### Analyse du sentiment dans 10 phrases comportant le mot "Femme"

In [ ]:
get_sentiment(" Pour les Travailleuses, contre le Féminisme lacs communistes ne voient on principe aucun intérêt à la conquête du bulletin de vote par « lu Française a>, pus plus que pur « le i' raneais », par • ia femme a> pas plus que pour l'homme")

In [ ]:
get_sentiment("quant aux rapports juridiques, ils etablissent sur l'égalité de tous les citoyens sans exception, c'est à dire de tous les habitants")

In [ ]:
get_sentiment("Et nous combattons le féminisme à cause de ces illusions et préjugés néfastes, nous nous efforçons de soustraire les travailleuses à son influence.")

In [ ]:
get_sentiment("quand les femmes françaises auront le droit de voter, on s'étonnera des obstacles qu'elles ont rencontrés pour l'obtenir")

In [ ]:
get_sentiment("Quant au point de savoir si le parlement de la bourgeoisie française accordera bientôt le droit de vote aux femmes, il nous paraît qu'il l'accordera au moment où il jugera que le vote féminin est nécessaire pour renforcer la majorité bourgeoise légale")

In [ ]:
get_sentiment("En Belgique, plus de la moitié de ta population est composée de femmes")

In [ ]:
get_sentiment("La Journée Internationale des Femmes du 8 mars, qui fut instituée à l'initiative de ce modèle de la femme révolutionnaire qu'était notre grande et vénérée Clara Zetldn, doit nous être une occasion nouvelle de poser et de résoudre le problème de la conquête des femmes travailleuses au Communisme")

In [ ]:
get_sentiment("La place de la femme est au foyer !, a déclaré Hitler")

In [ ]:
get_sentiment("Le libre accès à toutes les formes d'enseignement et l'égalité dans les conditions d'admission à tous les Etablissements d'enseignement : primaire, secondaire, supérieur, artistique, technique, etc")

In [ ]:
get_sentiment(" A ohaque fois que les élections approchent, l'on commence à s'agiter sur le problème du droit de vote des femmes")

### Analyse du sentiment dans 10 phrases comportant le mot "Droit"

In [ ]:
get_sentiment("Un grand nombre de femmes suivant les partis socialistes crurent que petit à petit par la lutte parlementaire, par la voie des réformes, elles parviendront à obtenir l'égalité des droits")

In [ ]:
get_sentiment("L'égalité des droits politiques")

In [ ]:
get_sentiment("Le droit à soustraire leurs enfants à toute éducation militariste")

In [ ]:
get_sentiment("Même les partis les moins partisans de ce droit de vote en période habituelle, sachant combien la' femme a pris aujourd'hui une place incontestée dans tous les domaines de la vie sociale, ne peuvent en période électorale ignorer ce problème, résolu depuis bien longtemps déjà, et en faveur du droit de vote des femmes, dans de nombreux pays démocratiques")

In [ ]:
get_sentiment("Le parti communiste qui a toujours eu une position claire sur ce problème, vient de faire déposer sur le bureau de la Chambre par les députés Alice Degeer, Bosson et Relecom, un projet de loi attribuant purement et simplement droit de vote aux femmes au même titre qu'aux hommes, projet qui ne se contente pas seulement d'être clair mais qui rapœ")

In [ ]:
get_sentiment("Partisans du droit des femmes, les libéraux pensent qu il faut donner à la femme une préparation générale qui lui permette de se guider elle-même dans la vie")

In [ ]:
get_sentiment("Le droit de suffrage féminin à se de- pré de l'électorat n'est d'ailleurs pas même en discussion, mais la Chambre à commencé à examiner l'électanat féminin à la province")

In [ ]:
get_sentiment("Parallèlement à cela, les femmes de la petite bourgeoisie forcent les portes des universités et des écoles, les suffragettes anglaises mènent une lutte courageuse pour le droit de vote des femmes")

In [ ]:
get_sentiment("Après la guerre,, certains pays ont accordé le droit de vote aux femmes, par exemple l'Allemagne, l'Angleterre, la Suède")

In [ ]:
get_sentiment("s'agit d'établir ici l'opportunité du droit de vote des femmes pour la province, pour faire ainsi un pas de plus vers le suffrage réellement universel, qui n'existe pas encore avec notre législation actuelle")

## Clustering des documents

In [ ]:
import collections
import os
import string
import sys
import nltk

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from timeout_decorator import timeout, TimeoutError
from threading import Timer
from nltk.tokenize import sent_tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

In [ ]:
import nltk

nltk.download('punkt')

## Vectoriser les documents à l'aide de TF-IDF

In [ ]:
## Vectoriser les documents à l'aide de TF-IDF
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

## Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

### Tests sur nos documents

In [ ]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
# Vecteur du document 0
tfidf_array[0]

In [ ]:
# Vecteur du document 1
tfidf_array[1]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

# définir un nombres de clusters

## Appliquer un algorithme de clustering sur les vecteurs TF-IDF des documents

In [ ]:
N_CLUSTERS = 2
# définir le nombre de clusters

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

In [ ]:
# Lecture du  contenu des clusters avec une limite de temps
def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# gestion  du  délai d'exécution
def timeout_handler():
    raise TimeoutError("Function execution timed out")

# Exploration des données brutes de chaque cluster
for cluster_label, files_in_cluster in clustering.items():
    print(f"Cluster {cluster_label} - Raw Data:")
    #pprint(files_in_cluster)
    print("\n")

    # Stocke du contenu de ces fichiers dans une liste
    content_list = []
    for txt in files_in_cluster:
        timer = Timer(5, timeout_handler)  # Définissez le délai en secondes
        try:
            timer.start()
            content = read_file_content(os.path.join(data_path, txt))
            content_list.append(content)
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")
            continue
        finally:
            timer.cancel()

    # Afficher le contenu de chaque texte dans le cluster
    for idx, text in enumerate(content_list):
        print(f"Text {idx + 1} in Cluster {cluster_label}:")
        print(text)
        print("\n")

# visualiser les clusters

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:20]

## Génerer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

## Segmentation en phrases du corpus complet et création d'un nouveau dossier 

In [ ]:
from nltk.tokenize import sent_tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode
from scipy.spatial.distance import cosine
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
infile = "../data/tmp/all_2.txt"
outfile = "../data/sents.txt"

In [ ]:
LIMIT = 100000

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content=content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")
print("Done")



In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

In [ ]:
bigram_phrases.vocab[key_]

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:200])

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
outfile = f"../data/newspapers.model"
model.save(outfile)

In [ ]:
model = Word2Vec.load("../data/newspapers.model")

In [ ]:
model.wv["femme"]

In [ ]:
model.wv["droit"]

### Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("femme", "droit")

In [ ]:
model.wv.similarity("droit", "gouvernement")

In [ ]:
model.wv.similarity("femme", "gouvernement")

In [ ]:
model.wv.similarity("femme", "homme")

In [ ]:
model.wv.similarity("vote", "femme")

In [ ]:
model.wv.similarity("femme", "tribunal")

In [ ]:
model.wv.similarity("femme", "travail")

### Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("femme", topn=35)

In [ ]:
model.wv.most_similar("droit", topn=35)

In [ ]:
model.wv.most_similar("homme", topn=15)


# on entraine notre modèle avec de nouvelles paramètre en mettant le 
##window=8
##min_count=9

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=70, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=8, # La taille du "contexte", ici 7 mots avant et après le mot observé
    min_count=9, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
model.wv["femme"]

In [ ]:
model.wv["droit"]

on calcule de nouveau la similarité

In [ ]:
model.wv.similarity("femme", "droit")

In [ ]:
model.wv.similarity("droit", "gouvernement")

In [ ]:
model.wv.similarity("femme", "gouvernement")

In [ ]:
model.wv.similarity("vote", "femme")

In [ ]:
model.wv.similarity("femme", "tribunal")

### Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("femme", topn=45)

In [ ]:
model.wv.most_similar("droit", topn=45)

In [ ]:
model.wv.most_similar("homme", topn=15)

## on entraine encore le modèle avec comme la taille de window=10 et le min_count=15

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 7 mots avant et après le mot observé
    min_count=15, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
model.wv["femme"]

In [ ]:
model.wv["droit"]

on calcule de nouveau la similarité

In [ ]:
model.wv.similarity("femme", "droit")

In [ ]:
model.wv.similarity("femme", "gouvernement")

In [ ]:
model.wv.similarity("droit", "gouvernement")

In [ ]:
model.wv.similarity("vote", "femme")

In [ ]:
model.wv.similarity("femme", "tribunal")

## Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("femme", topn=35)

In [ ]:
model.wv.most_similar("droit", topn=35)

In [ ]:
model.wv.most_similar("homme", topn=15)